## Waiting for Execution of Another Electron

Covalent parallelizes execution of tasks (electrons) when their inputs and outputs are independent. You might nonetheless need an electron to wait for another electron to finish execution before starting.

The Covalent `Electron` class provides a method, `wait_for()`, that explicitly defers execution until another electron has finished executing.

### Prerequisites

Define a lattice where an electron `task_A` needs an electron `task_B` to complete before starting, even though the input of `task_A` does not depend directly on the output of `task_B`.

In [1]:
import covalent as ct

@ct.electron
def task_A(args_A):
    ...
    return result_A

@ct.electron
def task_B(args_B):
    ...
    return result_B

### Procedure

Use the `electron.wait_for()` function to force `task_A` to delay start of execution until `task_B` is completed:

In [2]:
@ct.lattice
def workflow():
    result_B = task_B(args_B)
    result_A = task_A(args_A).wait_for(result_B)

In the following example, `task_3` waits for completion of `task_1` even though `task_3`'s execution does not directly depend on the output of `task_1`:

In [3]:
import covalent as ct

@ct.electron
def task_1(a):
    import time
    time.sleep(3)
    return a ** 2

@ct.electron
def task_2(x, y):
    return x * y

@ct.electron
def task_3(b):
    return b ** 3

@ct.lattice
def workflow():
    res_1 = task_1(2)
    res_2 = task_2(res_1, 3)
    res_3 = task_3(5).wait_for(res_1)

    return task_2(res_2, res_3)

dispatch_id = ct.dispatch(workflow)()
result = ct.get_result(dispatch_id, wait=True)
print(result)


Lattice Result
status: POSTPROCESSING_FAILED
result: None
input args: []
input kwargs: {}
error: Post-processing failed: None

start_time: 2023-01-29 22:13:53.585804
end_time: 2023-01-29 22:13:57.154784

results_dir: /Users/mini-me/agnostiq/covalent/doc/source/how_to/coding/results
dispatch_id: 1b46ca30-f9ed-4292-8707-bf8d4c02e567

Node Outputs
------------
task_1(0): 4
:parameter:2(1): 2
task_2(2): 12
:parameter:3(3): 3
task_3(4): 125
:parameter:5(5): 5
task_2(6): 1500



The `wait_for()` function can take a list of electrons rather than a single electron. The calling task waits until all electrons in the list are finished before executing.

In the following example, `task_3` waits for electrons 1a, 1b, and 1c before executing:

In [4]:
import covalent as ct
import time


@ct.electron
def task_1a(a):
    time.sleep(1)
    return a ** 2

@ct.electron
def task_1b(a):
    time.sleep(1)
    return a ** 3

@ct.electron
def task_1c(a):
    time.sleep(1)
    return a ** 4

@ct.electron
def task_2(x, y):
    return x * y

@ct.electron
def task_3(b):
    return b ** 3

@ct.lattice
def workflow():
    res_1a = task_1a(2)
    res_1b = task_1b(2)
    res_1c = task_1c(2)
    res_2 = task_2(res_1a, 3)
    res_3 = task_3(5).wait_for([res_1a, res_1b, res_1c])

    return task_2(res_2, res_3)

dispatch_id = ct.dispatch(workflow)()
result = ct.get_result(dispatch_id, wait=True)
print(result)


Lattice Result
status: POSTPROCESSING_FAILED
result: None
input args: []
input kwargs: {}
error: Post-processing failed: None

start_time: 2023-01-29 22:13:57.734608
end_time: 2023-01-29 22:13:58.945193

results_dir: /Users/mini-me/agnostiq/covalent/doc/source/how_to/coding/results
dispatch_id: f1a2d6f8-b7c0-4cfd-80f0-2b012aa7a419

Node Outputs
------------
task_1a(0): 4
:parameter:2(1): 2
task_1b(2): 8
:parameter:2(3): 2
task_1c(4): 16
:parameter:2(5): 2
task_2(6): 12
:parameter:3(7): 3
task_3(8): 125
:parameter:5(9): 5
task_2(10): 1500



The `wait_for()` function is especially useful when working with a dependent task that does not take any inputs and/or when you need to wait for classes that do not return an output.

### See Also

[Adding an Electron to a Lattice](./add_electron_to_lattice.ipynb)